** Measurements of electric power consumption in one household with a one-minute sampling rate over a period of almost 4 years.**

**PROBLEM DEFINATION**

> How well can we measure the electric power consumption in one household with a one-minute sampling rate over a period of almost 4 years with TIME SERIES

**DATA**

> This data is downloaded from kaggle notebook with the url https://www.kaggle.com/uciml/electric-power-consumption-data-set

> There are wo main dataset information

* This archive contains 2075259 measurements gathered between December 2006 and November 2010 (47 months).
 * Notes:
* (globalactivepower*1000/60 - submetering1 - submetering2 - submetering3) represents the active energy consumed every minute (in watt hour) in the household by electrical equipment not measured in sub-meterings 1, 2 and 3.

* The dataset contains some missing values in the measurements (nearly 1,25% of the rows). All calendar timestamps are present in the dataset but for some timestamps, the measurement values are missing: a missing value is represented by the absence of value between two consecutive semi-colon attribute separators. For instance, the dataset shows missing values on April 28, 2007.

**EVALUATION**

> we’ll explore and build time series forecasting models

**FEATURES**

> Hamoye has provided a guild on how to go about with the project

In [1]:
# Importing required libraries for data loading
import pandas as pd
import numpy as np

In [2]:
# Importing required libraries for data visualisation
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from statsmodels.tsa.arima_model import ARIMA 
from fbprophet import Prophet
from sklearn.model_selection import train_test_split

In [21]:
# Loading the dataset
power = pd.read_csv("../input/electric-power-consumption-data-set/household_power_consumption.txt", low_memory=False, sep=";",
                    header=0, infer_datetime_format=True, 
                    parse_dates={"datetime":[0,1]},index_col=["datetime"])
power.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [22]:
# Check the dataset dtype
power.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    object 
 1   Global_reactive_power  object 
 2   Voltage                object 
 3   Global_intensity       object 
 4   Sub_metering_1         object 
 5   Sub_metering_2         object 
 6   Sub_metering_3         float64
dtypes: float64(1), object(6)
memory usage: 126.7+ MB


In [23]:
# Checking for missing data
power.isna().sum()

Global_active_power          0
Global_reactive_power        0
Voltage                      0
Global_intensity             0
Sub_metering_1               0
Sub_metering_2               0
Sub_metering_3           25979
dtype: int64

In [24]:
# Checking the irregularrity in the missing values
count_na = 0
for values in power.values.tolist():
    for value in values:
        if value == '?':
            count_na += 1
        else:
            continue
            
print('We have {} missing values not captured by isna/isnull command'.format(count_na))

We have 155874 missing values not captured by isna/isnull command


In [25]:
# Rechecking for missing values again

# replace "?" to NaN
power.replace("?", np.nan, inplace = True)

#Let's recheck with the isna/isnull command
power.isna().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [26]:

#convert to float

for column in power.select_dtypes(include=['object']).columns:
    power[[column]] = power[[column]].astype('float')
    
power.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


In [27]:
# Time to fix the missing values
#using 0 to fill instead
power.fillna(value=0, inplace=True)

# Check it again
power.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

> We can see that all the missing values are been fixed




**NOW LETS CHECK IF ALL CODES ARE CORRECTLY EXECUTED**

In [28]:
# Now make a copy of the origibnal dataset to enable editting
power_new = power.copy()

In [29]:
# Now check the data again
power_new.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [30]:
# Lets also play aound the data
power_new.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06
mean,1.077950e+00,1.221658e-01,2.378249e+02,4.569827e+00,1.107879e+00,1.282265e+00,6.377598e+00
std,1.057642e+00,1.128556e-01,2.697024e+01,4.446361e+00,6.115669e+00,5.787271e+00,8.414871e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.040000e-01,4.600000e-02,2.388900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.780000e-01,1.000000e-01,2.409600e+02,2.600000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.520000e+00,1.920000e-01,2.428600e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01
max,1.112200e+01,1.390000e+00,2.541500e+02,4.840000e+01,8.800000e+01,8.000000e+01,3.100000e+01


Wow awesome, so we can proceed to the next stage of the data which is fitting the model
which would take a long of time

In [31]:
power_new['active_energy_consumed'] = power_new.apply(lambda row: row.Global_active_power  * 1000/60 - row.Sub_metering_1 - row.Sub_metering_2 - row.Sub_metering_3, axis=1)

In [32]:
# i think we should recheck for missing data before proceding
power_new.isna().sum()

Global_active_power       0
Global_reactive_power     0
Voltage                   0
Global_intensity          0
Sub_metering_1            0
Sub_metering_2            0
Sub_metering_3            0
active_energy_consumed    0
dtype: int64

In [33]:
#Let's check for any irregularities not captured by the isna/isnull function finally
count_na = 0
for values in power_new.values.tolist():
    for value in values:
        if value == '?':
            count_na += 1
        else:
            continue
#Print the result
print('Yey!!!!!!!!, Now we have {} missing values not captured by isna/isnull command'.format(count_na))

Yey!!!!!!!!, Now we have 0 missing values not captured by isna/isnull command


In [34]:
power_new.reset_index()
# Downsample 
# This reduces the number of samples in the data such that multiple data points are aggregated together.

daily_data = power_new.resample('D').mean()
daily_data = round(daily_data, 1)
daily_data = daily_data.interpolate(method='linear', limit_direction='forward')
print(daily_data.isna().sum())
 

Global_active_power       0
Global_reactive_power     0
Voltage                   0
Global_intensity          0
Sub_metering_1            0
Sub_metering_2            0
Sub_metering_3            0
active_energy_consumed    0
dtype: int64


In [36]:
adf_result = adfuller(daily_data['Global_active_power'])
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')
print(f'No. of lags used: {adf_result[2]}')
print(f'No. of observations used : {adf_result[3]}')
print('Critical Values:')

for k, v in adf_result[4].items():   
  print(f'   {k}: {v}')

ADF Statistic: -3.8480460389878584
p-value: 0.0024522718995536263
No. of lags used: 22
No. of observations used : 1419
Critical Values:
   1%: -3.434966750462565
   5%: -2.8635789736973725
   10%: -2.5678555388041384


In [37]:
# Downsample 
# This reduces the number of samples in the data such that multiple data points are aggregated together.

power_monthly = power.resample('M').mean()
power_monthly.head(3)
power_monthly.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [38]:
adf_result = adfuller(power_monthly['Global_active_power'])
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')
print(f'No. of lags used: {adf_result[2]}')
print(f'No. of observations used : {adf_result[3]}')
print('Critical Values:')

for k, v in adf_result[4].items():   
  print(f'   {k}: {v}')

ADF Statistic: -6.170640901509186
p-value: 6.818726060909473e-08
No. of lags used: 7
No. of observations used : 40
Critical Values:
   1%: -3.6055648906249997
   5%: -2.937069375
   10%: -2.606985625


In [39]:
# Time to reset index
power_monthly.reset_index()

,datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-31,1.900949,0.131362,241.397219,8.028496,1.248409,2.214584,7.408211
1,2007-01-31,1.545965,0.132670,240.894308,6.546622,1.264180,1.775851,7.383020
2,2007-02-28,1.401014,0.113631,240.507460,5.914276,1.180159,1.602282,6.703224
3,2007-03-31,1.318597,0.114744,240.508081,5.572854,1.361313,2.346819,6.504503
4,2007-04-30,0.814386,0.108542,218.768399,3.495977,0.974028,0.889282,4.386644
5,2007-05-31,0.985862,0.115343,235.178364,4.297464,1.696617,1.615860,5.139964
6,2007-06-30,0.825991,0.146250,238.637760,3.599963,1.381296,1.618958,4.371551
7,2007-07-31,0.665408,0.127107,236.973780,2.935493,0.964427,1.248499,3.468078
8,2007-08-31,0.763810,0.112761,237.819978,3.311035,0.812074,1.113598,5.050224
9,2007-09-30,0.969273,0.126005,239.413023,4.174417,1.223171,1.742523,5.240162


In [40]:
power_new= power_new.drop(columns = [ 'Global_reactive_power', 'Voltage','active_energy_consumed', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3',])
power_new.head(10)

,Global_active_power
datetime,
2006-12-16 17:24:00,4.216
2006-12-16 17:25:00,5.360
2006-12-16 17:26:00,5.374
2006-12-16 17:27:00,5.388
2006-12-16 17:28:00,3.666
2006-12-16 17:29:00,3.520
2006-12-16 17:30:00,3.702
2006-12-16 17:31:00,3.700
2006-12-16 17:32:00,3.668


In [41]:
power_new.describe()

,Global_active_power
count,2.075259e+06
mean,1.077950e+00
std,1.057642e+00
min,0.000000e+00
25%,3.040000e-01
50%,5.780000e-01
75%,1.520000e+00
max,1.112200e+01


In [42]:
power_new.head()

,Global_active_power
datetime,
2006-12-16 17:24:00,4.216
2006-12-16 17:25:00,5.360
2006-12-16 17:26:00,5.374
2006-12-16 17:27:00,5.388
2006-12-16 17:28:00,3.666


In [43]:
#A function for metrics
def metrics(y_true, y_pred):
  mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mse)
  return ("MAPE = {}, \n RMSE = {}".format(mape, rmse))

#using a function
def modelling(data, target, split_size, model):
  split_size = 100 * split_size
  split_size = int((len(data)/100) * split_size)
  data.interpolate(method='linear')
  """
  Using a function to do either a univariate or multivariate forecasting with fbprophet."""
  if model == 'univariate':
    data = data.reset_index()
    columns= data.columns[1:]
    X = data.drop(columns=(columns)) 
    y = data[target]
    X = X.rename(columns={"Date_Time": "ds"})
    y = pd.Series(data=y, name=("y"))
    x_train, y_train, x_test, y_test = X[:split_size], y[:split_size], X[split_size:], y[split_size:]
    train_model = pd.concat([x_train, y_train], axis=1, sort=False)
    test = pd.concat([x_test, y_test], axis=1, sort=False)
    model = Prophet(daily_seasonality=True)
    model.fit(train_model)
    forecast = model.predict(x_test)
    return (forecast, test)

  elif model == 'multivariate':
        
    data = data.reset_index()
    X = data.drop(columns=(target))
    y = data[target]
    X.rename(columns={"Date_Time": "ds"}, inplace=True)
    y = pd.Series(data=y, name=("y"))
    x_train, y_train, x_test, y_test = X[:split_size], y[:split_size], X[split_size:], y[split_size:]
    train_model = pd.concat([x_train, y_train], axis=1, sort=False)
    test = pd.concat([x_test['ds'], y_test], axis=1, sort=False)
    model = Prophet(daily_seasonality=True)
    for column in data.columns:
      if column != target and column != 'Date_Time':
        model.add_regressor(column)
    model.fit(train_model)
    forecast = model.predict(x_test)
    return (forecast, test)


In [46]:
power_new = power_new.reset_index()
power_new.head(10)

,datetime,Global_active_power
0,2006-12-16 17:24:00,4.216
1,2006-12-16 17:25:00,5.360
2,2006-12-16 17:26:00,5.374
3,2006-12-16 17:27:00,5.388
4,2006-12-16 17:28:00,3.666
5,2006-12-16 17:29:00,3.520
6,2006-12-16 17:30:00,3.702
7,2006-12-16 17:31:00,3.700
8,2006-12-16 17:32:00,3.668
9,2006-12-16 17:33:00,3.662


In [47]:
power_new.shape

power_new =power_new.rename(columns={"datetime": "ds", "Global_active_power": "y"}) 
power_new = power_new[["ds","y"]]
power_new.head(10)

,ds,y
0,2006-12-16 17:24:00,4.216
1,2006-12-16 17:25:00,5.360
2,2006-12-16 17:26:00,5.374
3,2006-12-16 17:27:00,5.388
4,2006-12-16 17:28:00,3.666
5,2006-12-16 17:29:00,3.520
6,2006-12-16 17:30:00,3.702
7,2006-12-16 17:31:00,3.700
8,2006-12-16 17:32:00,3.668
9,2006-12-16 17:33:00,3.662


In [49]:
def metrics(y_true, y_pred):
  mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mse)
  return ("MAPE = {}, \n RMSE = {}".format(mape, rmse))

In [48]:
#make predictions
forecast, test = modelling(data=daily_data, target='Global_active_power', split_size=0.7, model='univariate')

metric_df = forecast.set_index('ds')[['yhat']].join(test.set_index('ds')).reset_index()
metric_df.yhat = round(metric_df.yhat, 1)

#take the metrics
metrics(metric_df.y, metric_df.yhat)

ValueError: Dataframe must have columns "ds" and "y" with the dates and values respectively.

In [ ]:
power_new

In [ ]:
power_new

In [ ]:
#Modelling
model = Prophet()
#fit the model
model.fit(power_new)